# 1 Modelling the external data
## Case b) Constrained EV amplitude

**Author:** Hannu Parviainen<br>
**Last edited:** 26.5.2022

Here we model the external data constraining the EV amplitudes using theoretical amplitude ratios rather than the absolute EV amplitudes. Otherwise everything is like in the 1a notebook.

In [ ]:
%run src/init_threading_for_multiprocessing.py

In [ ]:
%matplotlib inline

In [ ]:
from multiprocessing import Pool
from numpy.random import permutation
from src import ExternalDataLPF, read_mcmc

In [ ]:
lpf = ExternalDataLPF('b')

In [ ]:
def lnposterior(pv):
    return lpf.lnposterior(pv)

pool = Pool(processes=50)

In [ ]:
how_to_start = 'start_from_scenario_a'
#how_to_start = 'start_from_scratch'
#how_to_start = 'continue'
npop = 200

if how_to_start is 'start_from_scratch':
    lpf.optimize_global(45000, npop, pool=pool, lnpost=lnposterior, vectorize=False)
    lpf.sample_mcmc(50, thin=10, repeats=1, save=False, pool=pool, lnpost=lnposterior, vectorize=False)
else:
    if how_to_start is 'continue':
        df = read_mcmc(f'results/{lpf.name}.nc')
    elif how_to_start is 'start_from_scenario_a':
        df = read_mcmc('results/01a_ext_emission_and_theoretical_ev.nc')
    else:
        raise NotImplementedError
    lpf.optimize_global(1, npop, pool=pool, lnpost=lnposterior, vectorize=False)
    sids = permutation(df.values.shape[0])[:lpf.de.n_pop]
    starting_population = df.values[sids, :-1].copy()
    lpf.sample_mcmc(50, thin=10, repeats=1, save=False, population=starting_population, 
                   pool=pool, lnpost=lnposterior, vectorize=False)

In [ ]:
lpf.sample_mcmc(5000, thin=50, repeats=10, save=True, pool=pool, lnpost=lnposterior, vectorize=False)

---

<center>&copy 2022 Hannu Parviainen</center>